<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [36]:
import requests
import numpy as np
import pandas as pd

In [92]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

import tensorflow as tf
from tensorflow.keras.preprocessing import sequence

import re
import numpy as np
import pandas as pd
import random
import sys
import os

In [37]:
from data_cleaning_toolkit_class import data_cleaning_toolkit

In [38]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [40]:
dctk = data_cleaning_toolkit()

# use regex to clean documents
df_toc["text"] = df_toc.text.apply(lambda doc: dctk.clean_data(doc))

In [63]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379,
1,AS YOU LIKE IT,14380,17171,as you like it\r \r \r dramatis personae\r \rd...
2,THE COMEDY OF ERRORS,17172,20372,the comedy of errors\r \r \r \r contents\r \r ...
3,THE TRAGEDY OF CORIOLANUS,20373,30346,the tragedy of coriolanus\r \r dramatis person...
4,CYMBELINE,30347,30364,cymbeline\r laud we the gods\r and let our cro...


In [59]:
# eliminate rows without any data
df = df_toc[df_toc['text'] != '']

In [65]:
# remove leftover '\r'
def remove_r(row):
    row = row.replace('\r', '')
    return row

In [66]:
# remove leftover '\r'
df['text'] = df['text'].apply(remove_r)

<ipython-input-66-95a5bd64e4ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(remove_r)


In [73]:
# reset index to forget about elimitaed rows
df = df.reset_index()

In [75]:
# move docs to array
data = df["text"].values

In [77]:
doc_len = 200
dctk.create_char_sequenes(data, doc_len)

sequences:  2645960


In [78]:
# create X and Y split 
x, y = dctk.create_X_and_Y()

In [79]:
dtm = dctk.sequences

In [85]:
int_seq_vect = dctk.sequences[0]
for int_ in int_seq_vect:
    print (dctk.int_char[int_])

a
s
 
y
o
u
 
l
i
k
e
 
i
t
 
 
 
d
r
a
m
a
t
i
s
 
p
e
r
s
o
n
a
e
 
d
u
k
e
 
l
i
v
i
n
g
 
i
n
 
e
x
i
l
e
f
r
e
d
e
r
i
c
k
 
h
i
s
 
b
r
o
t
h
e
r
 
a
n
d
 
u
s
u
r
p
e
r
 
o
f
 
h
i
s
 
d
o
m
i
n
i
o
n
s
a
m
i
e
n
s
 
l
o
r
d
 
a
t
t
e
n
d
i
n
g
 
o
n
 
t
h
e
 
b
a
n
i
s
h
e
d
 
d
u
k
e
j
a
q
u
e
s
l
e
 
b
e
a
u
 
a
 
c
o
u
r
t
i
e
r
 
a
t
t
e
n
d
i
n
g
 
u
p
o
n
 
f
r
e
d
e
r
i
c
k
c
h
a
r


In [86]:
dctk.n_features

28

In [87]:
x.shape

(2645960, 200, 28)

In [88]:
y.shape

(2645960, 28)

In [90]:
def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array
    """
    # convert preds to array 
    preds = np.asarray(preds).astype('float64')
    # scale values 
    preds = np.log(preds) / temperature
    # exponentiate values
    exp_preds = np.exp(preds)
    # this equation should look familar to you (hint: it's an activation function)
    preds = exp_preds / np.sum(exp_preds)
    # Draw samples from a multinomial distribution
    probas = np.random.multinomial(1, preds, 1)
    # return the index that corresponds to the max probability 
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    """"
    Function invoked at end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # prevent out of bounds index error 
    max_index = len(text) - dctk.maxlen - 1
    
    # randomly sample an index between 0 and the max 
    start_index = random.randint(0, max_index)
    
    generated = ''
    
    # randomly sample a portion of text from the joined articles, text
    # this sentence will have dctk.maxlen many chars 
    sentence = text[start_index: start_index + dctk.maxlen]
    
    # add that random snippit to the generated var
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # iterate through 400 chars 
    for i in range(400):
        
        # create an input vector with the shape that the model expects
        x_dims = (1, dctk.maxlen, dctk.n_features)
        
        # all default values will be zero
        x_pred = np.zeros(x_dims)
        
        # iterate through all the chars in sentence
        for t, char in enumerate(sentence):
            # numerically encode sentence
            x_pred[0, t, dctk.char_int[char]] = 1
            
        # use model to generate text 
        preds = model.predict(x_pred, verbose=0)[0]
        
        # extract index from predictions 
        next_index = sample(preds)
        
        # get next char based on the most probable char given by sample(preds)
        next_char = dctk.int_char[next_index]
        
        # append model generated chars to our started sentence 
        sentence = sentence[1:] + " " + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
        
        # iterate through all the chars in sentence
        for t, char in enumerate(sentence):
            # numerically encode sentence
            if t != doc_len:
                x_pred[0, t, dctk.char_int[char]] = 1
            else:
                break
    print()

In [93]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [95]:
# build another model for our task for forecasting what text should follow from our seed string 
model = Sequential()

# hidden layer 1 
model.add(LSTM(128, 
               input_shape=(dctk.maxlen, dctk.n_features), # think of input_shape as implicitly declaring the input layer 
               return_sequences=True)) # set to true whenever using 2 or more LSTM layers 

# hidden layer 2 
model.add(LSTM(64))

# this is our output layer
# recall that n_features = num of nodes in output layer 
model.add(Dense(dctk.n_features, 
                activation='softmax'))

# notice that we are using categorical_crossentropy this time around - why?
model.compile(loss='categorical_crossentropy', 
              optimizer='adam')

# fit the model
# x and y are pretty large, consider sub-sampling
model.fit(x, y,
          batch_size=256,
          epochs=1,
          callbacks=[print_callback])

 1080/10336 [==>...........................] - ETA: 101:53:51 - loss: 2.6681

KeyboardInterrupt: 

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN